# Gurobi

In [3]:
from gurobipy import *
import pandas as pd
import numpy as pd
import matplotlib.pyplot as plt

## example1

max $$ x+y+2z$$

s.t. $$
    x+2y+3z<=4\\
    x+y>=1
    $$

In [7]:
model=Model("example1")

x=model.addVar(vtype=GRB.BINARY,name='x')
y=model.addVar(vtype=GRB.BINARY,name='y')
z=model.addVar(vtype=GRB.BINARY,name='z')

model.setObjective(x+y+2*z,GRB.MAXIMIZE)

model.addConstr(x+2*y+3*z<=4,'c1')
model.addConstr(x+y>=1,'c2')

model.optimize()

for v in model.getVars():
    print(f"{v.VarName}:{v.x}")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x:1.0
y:0.0
z:1.0


## food

In [14]:

### addVars
### addConstrs

model=Model("food_supply")
category,minNutrition,maxNutrition=multidict({
        'calories':[1800,2000],
        'protein':[91,GRB.INFINITY],
        'fat':[0,65],
        'sodium':[0,1779]
    })
foods,cost=multidict({
    'hamburger':2.49,
    'chicken':2.89,
    'hotdog':1.50,
    'fries':1.89,
    'macaroni':2.09,
    'pizza':1.99,
    'salad':2.49,
    'milk':0.89,
    'ice cream':1.59
})
nutritionValue={
    ('hamburger','calories'):410,
    ('hamburger','protein'):24,
    ('hamburger','fat'):26,
    ('hamburger','sodium'):730,

    ('chicken','calories'):420,
    ('chicken','protein'):32,
    ('chicken','fat'):10,
    ('chicken','sodium'):1190,

    ('hotdog','calories'):560,
    ('hotdog','protein'):20,
    ('hotdog','fat'):32,
    ('hotdog','sodium'):1800,

    ('fries','calories'):380,
    ('fries','protein'):4,
    ('fries','fat'):19,
    ('fries','sodium'):270,

    ('macaroni','calories'):320,
    ('macaroni','protein'):12,
    ('macaroni','fat'):10,
    ('macaroni','sodium'):930,

    ('pizza','calories'):320,
    ('pizza','protein'):15,
    ('pizza','fat'):12,
    ('pizza','sodium'):820,

    ('salad','calories'):320,
    ('salad','protein'):31,
    ('salad','fat'):12,
    ('salad','sodium'):1230,

    ('milk','calories'):100,
    ('milk','protein'):8,
    ('milk','fat'):2.5,
    ('milk','sodium'):125,

    ('ice cream','calories'):330,
    ('ice cream','protein'):8,
    ('ice cream','fat'):10,
    ('ice cream','sodium'):180
}
buy=model.addVars(foods,name='buy')

model.setObjective(buy.prod(cost),GRB.MINIMIZE)#对应相乘

model.addConstrs((quicksum(nutritionValue[f,c]*buy[f] for f in foods)==[minNutrition[c],maxNutrition[c]] for c in category),'_')

model.optimize()

print(f"{model.ObjVal},{buy.keys}")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0xe211d8b6
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [7e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.182886111e+01
11.828861111111111,<bound method tupledict.keys of {'hamburger': <gurobi.Var buy[hamburger] (value 0.6045138888888888)>, 'chicken': <gurobi.Var buy[chicken] (value 0.0)>, 'hotdog': <gurobi.Var buy[hotdog] (value 0.0)>, 'fries': <gu

## 参数

`TimeLimit`   `设定时间`
`SolutionLimit` 可行解数量

In [ ]:
#!/usr/bin/env python3.7

import sys
import gurobipy as gp


if len(sys.argv) < 2:
    print('Usage: params.py filename')
    sys.exit(0)


# Read model and verify that it is a MIP
m = gp.read(sys.argv[1])
if m.IsMIP == 0:
    print('The model is not an integer program')
    sys.exit(1)

# Set a 2 second time limit
m.Params.TimeLimit = 2

# Now solve the model with different values of MIPFocus
bestModel = m.copy()
bestModel.optimize()
for i in range(1, 4):
    m.reset()
    m.Params.MIPFocus = i
    m.optimize()
    if bestModel.MIPGap > m.MIPGap:
        bestModel, m = m, bestModel  # swap models

# Finally, delete the extra model, reset the time limit and
# continue to solve the best model to optimality
del m
bestModel.Params.TimeLimit = float('inf')
bestModel.optimize()
print('Solved with MIPFocus: %d' % bestModel.Params.MIPFocus)

## 广义约束

In [18]:
#m.addGenConstrMax 
model=Model("Max")
x=model.addVar(obj=1,vtype=GRB.CONTINUOUS,name='x')
y=model.addVar(obj=1,vtype=GRB.CONTINUOUS,name='y')
z=model.addVar(obj=1,vtype=GRB.CONTINUOUS,name='z')

# model.addGenConstrMax(z,[x,y,3],name='maxconstr')
# model.addGenConstrMin(z,[x,y,3],name='min')

#z==max(min)[x,y,3]

model.addGenConstrAbs(x,y,name='abs')
model.optimize()


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x532d5078
Model has 1 general constraint
Variable types: 3 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


### And Or Indicator Range

In [31]:
#And
model=Model("AndOr")

x=model.addVar(obj=1,vtype=GRB.CONTINUOUS,name='x')
y=model.addVar(obj=1,vtype=GRB.CONTINUOUS,name='y')
z=model.addVar(obj=1,vtype=GRB.CONTINUOUS,name='z')
model.addGenConstrAnd(z,[x,y])

# model.addConstr(z==and_(x,y),'and')

# model.addGenConstrOr(z,[x,y],'or')
# model.addConstr(z==or_(x,y),'or')

# model.addGenConstrIndicator(z,True,x+y,GRB.LESS_EQUAL,4,'indicator')
# model.addConstr((z==1)>>(x+y<=4),'ind')

# model.addRange(x+y+z,5,10,'R')
# model.addConstr(x+y+z==[5,12])


model.write("example.lp")

## 列生产

## 波次优化 BatchOpt